#### 1. Instalação de Bibliotecas

In [0]:
pip install kafka-python

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install translate

Python interpreter will be restarted.
  Using cached translate-3.6.1-py2.py3-none-any.whl (12 kB)
  Using cached lxml-5.3.0-cp39-cp39-manylinux_2_28_x86_64.whl (5.0 MB)
  Using cached libretranslatepy-2.1.1-py3-none-any.whl (3.2 kB)
Python interpreter will be restarted.


#### 2. Imports

In [0]:
from kafka import KafkaProducer
import json
import requests
from translate import Translator
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.utils import AnalysisException

#### 3. Constantes

In [0]:
topico = 'meu-topico'

translator = Translator(to_lang="en", from_lang='pt')
# api_key = "844629b1a5d44eb7938b174b6285a905"
api_key = 'fac1628b04464126940239f493d9eda5'

schema = StructType([
        StructField("source", StructType([
            StructField("id", StringType(), True),
            StructField("name", StringType(), True)
        ]), True),
        StructField("author", StringType(), True),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("url", StringType(), True),
        StructField("urlToImage", StringType(), True),
        StructField("publishedAt", StringType(), True),
        StructField("content", StringType(), True)])

#### 4. Busca de dados da API newsapi

In [0]:
def carregamento(palavras_chave: list[str]):
    searchIn = 'title,description,content'
    news_translation = list(map(translator.translate, palavras_chave))

    df_articles = spark.createDataFrame([], schema)

    for noticia in set(palavras_chave + news_translation):
        url = ('https://newsapi.org/v2/everything?'
        f'q=("{noticia}")&'
        f'searchIn={searchIn}&'
        'sortBy=relevancy&'
        f'apiKey={api_key}')

        response = requests.get(url).json()
        if response['status'] == 'ok':
                df_articles = df_articles.union(spark.createDataFrame(response['articles'], schema))
    return df_articles

def processamento_raw(df):
  df = df.dropDuplicates(['url'])
  return df

#### 5. Envia as mensagens

In [0]:
# Função de serialização para converter em bytes
def serializer(message):
    return json.dumps(message).encode('utf-8')

# Criando o produtor
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],  # Endereço do Kafka
    value_serializer=serializer  # Serializando mensagens
)

# Enviando mensagens
def enviar_mensagens(dataFrame):
    for row in dataFrame.collect():
        producer.send(topico, value=row.asDict())
        print(f"Mensagem enviada: {row.asDict()}")

palavras_chave = ['câncer']

df = carregamento(palavras_chave)
df_processada = processamento_raw(df)

enviar_mensagens(df_processada)

# Fecha o produtor
producer.close()

Mensagem enviada: {'source': Row(id=None, name='Purepeople.com.br'), 'author': 'Purepeople BR', 'title': "Gente? Madonna faz ensaio fotográfico dramático em velório do irmão e agita a web: 'Quem morreu foi ele'", 'description': 'Se essa moda pega... A rainha do pop simplesmente resolveu tirar fotos em preto e branco no velório do irmão mais novo, Christopher Ciccone. Ele faleceu na semana passada, vítima de um câncer.', 'url': ' https://www.purepeople.com.br/noticia/irmao-de-madonna-morre-e-cantora-faz-ensaio-fotografico-em-preto-e-branco-web-reage-quem-morreu-foi-ele_a397493/1 ', 'urlToImage': 'https://p2.trrsf.com/image/fget/cf/1200/630/middle/images.terra.com/2024/10/12/626425138-4531625-apos-morte-do-irmao-mais-novo-madonna-c-1400x823-3.jpg', 'publishedAt': '2024-10-12T01:44:09Z', 'content': 'Madonna , que fez um show histórico no Rio de Janeiro este ano, causou polêmica nas redes sociais nesta sexta-feira (11). A rainha do pop, que tem uma conexão pouco conhecida com Bruna Marquez